# AMOS 2024 - Heatwave case study using BARRA2 reanalysis

##### In this notebook we demonstrate the use of BARRA2 data in the NCI Data Collection to explore the NSW heatwave of January 2017.

##### More information on BARRA2 data: https://opus.nci.org.au/pages/viewpage.action?pageId=264241166

##### Before using this notebook, users must join ob53 project via, https://my.nci.org.au/mancini/project/ob53/join

In [ ]:
import os, sys
os.chdir("/g/data/hd50/chs548/BARRA2_evaluation/jt/notebooks/")
import loaddata
from datetime import datetime as dt
# Here we will use the LOADDATA module to simply the loading of the data
import loaddata
import matplotlib.pyplot as plt
import matplotlib as mpl
import cartopy.crs as ccrs
import numpy as np
import xarray as xr

#### Explore the data sets - what's available and what they are

In [ ]:
# What experiments are available in BARRA2 at this time?
loaddata.list_experiments("BARRA2")

# At this time, we have BARRA-R2 12 km reanalysis data. 
# Later this year, the BARRA-RE2 ensemble and BARRA-C2 4.4 km reanalysis data will be available.

In [ ]:
# What hourly variables are available?
_ = loaddata.list_barra2_variables("BARRA-R2", "1hr")

In [ ]:
# For heatwave case study, we will look at the screen-level temperature, near-surface wind vector and low level cloud cover
# First check they are what they are, and explore different variable options
_ = loaddata.whatis("1hr", "tas")

In [ ]:
_ = loaddata.whatis("1hr", "tasmax")

In [ ]:
_ = loaddata.whatis("1hr", "uas")

In [ ]:
_ = loaddata.whatis("1hr", "uasmax")

In [ ]:
_ = loaddata.whatis("1hr", "vas")

In [ ]:
_ = loaddata.whatis("1hr", "cll")

In [ ]:
_ = loaddata.whatis("fx", "orog")

In [ ]:
_ = loaddata.whatis("fx", "sftlf")

#### Function to load the data

In [ ]:
# We will use hourly maximum temperature (tasmax), hourly inst u and v (uas, vas), and hourly mean low-level cloud cover (cll)
# But feel free to explore other variables of your interests, such as pressure-level variables such as va850, ta850
help(loaddata.load_barra2_data)

#### What's the temperature in the region around Sydney during January 2017?

##### Plot the spatial mean of tasmax over during the few months and pick a smaller time period to examine

In [ ]:
loc = (-33.8688, 151.2093)
latmin_c = loc[0] - 1
latmax_c = loc[0] + 1
lonmin_c = loc[1] - 1
lonmax_c = loc[1] + 1
tstart = dt(2016, 12, 1)
tend = dt(2017, 4, 1)

# Load tasmax data
ds_tasmax = loaddata.load_barra2_data("BARRA-R2", "1hr", "tasmax", 
                                      tstart=tstart, tend=tend,
                                      latrange=(latmin_c, latmax_c),
                                      lonrange=(lonmin_c, lonmax_c))
# Convert to degC
ds_tasmax['tasmax'] = ds_tasmax['tasmax'] - 273.15
# TODO: update units attribute
(NT, NY, NX) = ds_tasmax['tasmax'].shape

# Load land sea mask
ds_lsm = loaddata.load_barra2_data("BARRA-R2", "fx", "sftlf",
                                   latrange=(latmin_c, latmax_c),
                                      lonrange=(lonmin_c, lonmax_c))

# Compute spatial mean of tasmax over land, over the smaller subdomain around Sydney
mask = (np.tile(ds_lsm['sftlf'].values, (NT,1,1)) >= 100)
da_tasmax_spatial_av = ds_tasmax['tasmax'].where(mask).mean(dim=['lat', 'lon'])

In [ ]:
fig = plt.figure(figsize=(10, 3))

# 
t0 = dt(2017, 2, 7)
t1 = dt(2017, 2, 14)
#
# First subfigure showing the temperature changes during the whole period
# 
ax = plt.subplot(1, 2, 1)
da_tasmax_spatial_av.plot.line()
plt.axvline(x=t0, color='r')
plt.axvline(x=t1, color='r')
#
# Second subfigure showing the temperature changes 
# 
ax = plt.subplot(1, 2, 2)
da_tasmax_spatial_av.sel(time=slice(t0, t1)).plot.line()

#### Define domain and time of interests

In [ ]:
# Define domain and time of interests as per above
tstart = dt(2017, 2, 8)
tend = dt(2017, 2, 14)

latmin_r = loc[0] - 5
latmax_r = loc[0] + 5
lonmin_r = loc[1] - 5
lonmax_r = loc[1] + 5

#### Load the data sets

In [ ]:
# Load the orography data
ds_orog = loaddata.load_barra2_data("BARRA-R2", "fx", "orog", 
                                    latrange=(latmin_r, latmax_r),
                                      lonrange=(lonmin_r, lonmax_r))
# Load land sea mask
ds_lsm = loaddata.load_barra2_data("BARRA-R2", "fx", "sftlf",
                                   latrange=(latmin_r, latmax_r),
                                      lonrange=(lonmin_r, lonmax_r))

In [ ]:
ds_tasmax = loaddata.load_barra2_data("BARRA-R2", "1hr", "tasmax", 
                                      tstart=tstart, tend=tend,
                                      latrange=(latmin_r, latmax_r),
                                      lonrange=(lonmin_r, lonmax_r))
ds_uas = loaddata.load_barra2_data("BARRA-R2", "1hr", "uas", 
                                      tstart=tstart, tend=tend,
                                      latrange=(latmin_r, latmax_r),
                                      lonrange=(lonmin_r, lonmax_r))
ds_vas = loaddata.load_barra2_data("BARRA-R2", "1hr", "vas", 
                                      tstart=tstart, tend=tend,
                                      latrange=(latmin_r, latmax_r),
                                      lonrange=(lonmin_r, lonmax_r))
ds_cll = loaddata.load_barra2_data("BARRA-R2", "1hr", "cll", 
                                      tstart=tstart, tend=tend,
                                      latrange=(latmin_r, latmax_r),
                                      lonrange=(lonmin_r, lonmax_r))

#### Pre-process the data for plotting

In [ ]:
# Convert temperature from K to degC
ds_tasmax['tasmax' ] = ds_tasmax['tasmax'] - 273.15
(NT, NY, NX) = ds_tasmax['tasmax'].shape

In [ ]:
# Combine uas and vas into a single xr.Dataset object
ds_uv = xr.merge([ds_uas['uas'], ds_vas['vas']])
# Thin the data horizontally for plotting purposes, to avoid having very densed wind vectors
ds_uv_subsampled = ds_uv.isel(lat=range(0, NY, 7), lon=range(0, NX, 7)).compute()

In [ ]:
# Compute spatial mean of tasmax over the smaller subdomain around Sydney
ds_lsm_c = ds_lsm.sel(lat=slice(latmin_c, latmax_c), lon=slice(lonmin_c, lonmax_c))
mask_c = (np.tile(ds_lsm_c['sftlf'], (NT,1,1)) >= 100)
ds_tasmax_c = ds_tasmax.sel(lat=slice(latmin_c, latmax_c), lon=slice(lonmin_c, lonmax_c))
da_tasmax_spatial_av = ds_tasmax_c['tasmax'].where(mask_c).mean(dim=['lat', 'lon'])

#### Plot the heatwave with a series of 3-panels

##### Each 3-panel shows,
##### Left: Timeseries of tasmax around Sydney
##### Middle: Spatial maps of tasmax and 10 m wind
##### Right: Spatial maps of cloud cover

In [ ]:
for time_step in range(0, NT, 3):
    
    # Plotting for this time step
    t = ds_tasmax['time'][time_step]
    
    # Set up the figure object
    fig = plt.figure(figsize=(14, 4))
    
    #
    # First subfigure showing the timeseries of spatial mean tasmax
    #
    ax1 = plt.subplot(1, 3, 1)
    da_tasmax_spatial_av.plot.line()
    ax1.plot(t.data, da_tasmax_spatial_av.data[time_step], 'or')
    # label the time step
    ax1.set_title(t.data)
    
    #
    # Second subfigure plotting temperature and wind vectors
    #
    ax2 = plt.subplot(1, 3, 2)
    # Plot tasmax as background
    ds_tasmax.sel(time=t, method='nearest')['tasmax'].plot(vmin=0, vmax=40, cmap=mpl.cm.RdBu_r, cbar_kwargs={"shrink": 0.5})
    # Plot the 10m wind vector
    ds_uv_subsampled.sel(time=t, method='nearest').plot.quiver(x='lon', y='lat', u='uas', v='vas', color='blue')
    # Plot surface altitude as contour
    ds_orog['orog'].plot.contour(levels=4, colors='k')
    # Indicate where is Sydney
    ax2.plot(loc[1], loc[0], 'xr', markersize=15)
    
    #
    # Last subfigure plotting low level cloud
    #
    ax3 = plt.subplot(1, 3, 3)
    # Plot cloud 
    ds_cll['cll'].sel(time=t, method='nearest').plot(vmin=0, vmax=100, cmap=mpl.cm.Greys_r, cbar_kwargs={"shrink": 0.5})
    # Plot surface altitude as contour
    ds_orog['orog'].plot.contour(levels=4, colors='b')
    # Indicate where is Sydney
    ax3.plot(loc[1], loc[0], 'xr', markersize=15)
    
    fig.tight_layout()
    
# this may take a while!